In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from mpl_toolkits.mplot3d import Axes3D
from heuristi_plot_1 import plot as hplot

path = '../output_data/'

In [ ]:
for i in [1, 3, 5, 6, 7, 8, 9]:
#for i in [5, 8, 9]:
    print(f'phi = 0.{i}')
    with pd.HDFStore(path + 'X0c.h5') as store:
        trj = store.select('dat_0', where=f'phi=.{i}')
    with pd.HDFStore(path + 'X0c.h5') as store:
        trj2 = store.select('dat_2', where=f'phi=.{i}')

    trj2.index.names = trj.index.names

    for trj_i in [trj, trj2]:
        trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
        trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
    trj = trj.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
    trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])

    dfN = pd.DataFrame(index=trj.index.levels[0])

    fig = plt.figure(figsize=(10, 3))
    hplot(fig, trj, trj2, thresh_time=9.)

    fig.savefig(f'fitted_initials_evaluation0{i}.pdf')

In [ ]:
cond = pd.DataFrame(index=trj.index)
cond['a'] = trj['K_c']/trj['K_d'] - 0.15276374882897656
cond['b'] = trj['i_c']
cond['c'] = trj['r_c']

# cond['d'] = np.sqrt((cond['a'])**2)
# cond['d'].unstack('sample').loc[:9].sum(axis=0)

In [ ]:
trj.head()

In [ ]:
t_max=9.

# calculate point distance
cond['d'] = np.sqrt((cond['a'])**2)

# sum over point distance for each trajectory
dfN['distance'] = cond['d'].unstack('sample').loc[:t_max].sum(axis=0)

# map distances to interval (0,1]
dfN['inv_distance'] = 1./(1. + dfN['distance'])

# use 1/distance as weight
dfN['weight'] = 1/cond['d'].unstack('sample').loc[:t_max].sum(axis=0)
dfN['weight'] = dfN['weight']/sum(dfN['weight'].values)

for s in dfN.index.values:
    cond.loc[s, 'inv_distance'] = dfN.loc[s, 'inv_distance']
dfN['weight'].sum()
cond.head()

In [ ]:
from collections import Counter
fig, axes = plt.subplots(ncols=3, figsize=(8, 2.8))

def plot_cond(df, c, ax, t_max, thresh=0.5):
    tmp = df
    tmp1 = tmp[[c]].loc[tmp['inv_distance']<thresh]
    tmp2 = tmp[[c]].loc[tmp['inv_distance']>thresh]
    tmp1.unstack('sample').loc[:t_max].plot(ax=ax, alpha=.03, legend=False, color=['k'])
    tmp2.unstack('sample').loc[:t_max].plot(ax=ax, legend=False, alpha=.07, color=['orange'])
    
thresh=0.5

plot_cond(cond, 'a', axes[0], 20, thresh=thresh)
plot_cond(cond, 'b', axes[1], 20, thresh=thresh)

dfN['inv_distance'].loc[dfN['inv_distance']>thresh].plot.hist(ax=axes[2], color=['orange'], alpha=0.9, bins=0.1*np.arange(5,11))
dfN['inv_distance'].loc[dfN['inv_distance']<thresh].plot.hist(ax=axes[2], color=['k'], alpha=0.2, bins=0.1*np.arange(0,6))

for ax in axes[:2]:
    ax.set_xlim([0,20])
    ax.set_xlabel('time')
    ax.axvline(t_max, color='k')

axes[0].set_ylabel(r'$K_c/K_d - K_{c0}/K_{d0}$')
axes[1].set_ylabel('fraction of clean investment')
axes[1].set_ylim((0,1.03))
axes[2].set_ylabel('count')
axes[2].set_xlabel(r'$1/(1+d)$')
axes[2].set_xlim([0,1])

fig.tight_layout()
fig.savefig('initial_condition_fitted.pdf')

In [ ]:
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_0', where='phi=.8')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where='phi=.8')
    
trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])


In [ ]:
trj

In [ ]:
# setting parameters
thresh_time = 10.
thresh_indicator = 'i_c'
thresh_value = 0.5
t_max = 40.
indicators = ['i_c', 'G']
t_scatt = [20., t_max]
cond = pd.DataFrame(index=trj.index)
cond2 = pd.DataFrame(index=trj2.index)

for s, t in trj.index:
    print(s, t)
    cond.loc[(s, t), 
             thresh_indicator] = trj.loc[(s, thresh_time), thresh_indicator]

# for s in trj2.index.levels[0]:
#     tmp = trj2.loc[(s, thresh_time), thresh_indicator]
#     cond2.loc[(s, trj2.index.levels[1].values.tolist()),
#               thresh_indicator] = tmp 


In [ ]:
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_0', where='phi=.8')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where='phi=.8')
    
trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])


# setting parameters
thresh_time = 10.
thresh_indicator = 'i_c'
thresh_value = 0.5
t_max = 40.
indicators = ['i_c', 'G']
t_scatt = [20., t_max]
cond = pd.DataFrame(index=trj.index)
cond2 = pd.DataFrame(index=trj2.index)

In [ ]:
trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])

In [ ]:
i=5
print(f'phi = 0.{i}')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj = store.select('dat_0', where=f'phi=.{i}')
with pd.HDFStore(path + 'X0c.h5') as store:
    trj2 = store.select('dat_2', where=f'phi=.{i}')

trj2.index.names = trj.index.names

for trj_i in [trj, trj2]:
    trj_i.index = trj_i.index.droplevel(['eps', 'phi', 'test'])
    trj_i = trj_i.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj = trj.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj2 = trj2.reset_index().drop_duplicates(['sample', 'tstep']).set_index(['sample', 'tstep'])
trj.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

for obs, color in zip(['r_c', 'r_d'], ['blue', 'black']):
    print(obs)
    trj_plt = trj[[obs]].unstack('sample').plot(ax=ax, color=color, legend=False, alpha=0.1)

ax.set_xlim([5, 12.5])
ax.set_ylim([0.07, 0.08])

In [ ]:
trj.loc[trj['r_c']>trj['r_d']][['r_c', 'r_d']].reset_index().groupby('sample')